<a href="https://colab.research.google.com/github/cesar20c190/CONSULTADEF/blob/main/Acesso_DPB_Tx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json

In [57]:
url = "https://api-publica.datajud.cnj.jus.br/api_publica_tjba/_search"
api_key = "APIKey cDZHYzlZa0JadVREZDJCendQbXY6SkJlTzNjLV9TRENyQk1RdnFKZGRQdw==" # Chave pública

In [58]:
# Aqui trocamos o filtro da classe pelo código do município IBGE
payload = json.dumps({
  "size": 10000,
  "query": {
    "match": {
      "orgaoJulgador.codigoMunicipioIBGE": 2931350
    }
  },
  "sort": [
    {
      "dataAjuizamento": {
        "order": "desc"
      }
    }
  ]
})

headers = {
  'Authorization': api_key,
  'Content-Type': 'application/json'
}

response = requests.post(url, headers=headers, data=payload)
dados_dict = response.json()

# Mostra quantos processos vieram
print(f"Total de processos: {len(dados_dict['hits']['hits'])}")

Total de processos: 10000


In [59]:
display(dados_dict['hits']['hits'][10]) # Exemplo


{'_index': 'api_publica_tjba',
 '_id': 'TJBA_G1_80016990920258050256',
 '_score': None,
 '_source': {'numeroProcesso': '80016990920258050256',
  'classe': {'codigo': 279, 'nome': 'Inquérito Policial'},
  'sistema': {'codigo': 1, 'nome': 'Pje'},
  'formato': {'codigo': 1, 'nome': 'Eletrônico'},
  'tribunal': 'TJBA',
  'dataHoraUltimaAtualizacao': '2025-03-20T12:26:00.102Z',
  'grau': 'G1',
  '@timestamp': '2025-03-21T18:37:20.387277566Z',
  'dataAjuizamento': '2025-03-14T00:00:00.000Z',
  'movimentos': [{'complementosTabelados': [{'codigo': 2,
      'valor': 2,
      'nome': 'sorteio',
      'descricao': 'tipo_de_distribuicao_redistribuicao'}],
    'codigo': 26,
    'nome': 'Distribuição',
    'dataHora': '2025-03-14T00:00:00.000Z'},
   {'codigo': 12281,
    'nome': 'Comunicação eletrônica',
    'dataHora': '2025-03-14T00:00:00.000Z'}],
  'id': 'TJBA_G1_80016990920258050256',
  'nivelSigilo': 0,
  'orgaoJulgador': {'codigoMunicipioIBGE': 2931350,
   'codigo': 76526,
   'nome': '2ª VARA 

In [60]:
if dados_dict['hits']['total']['value'] > 0 and len(dados_dict['hits']['hits']) > 0:
    processo = dados_dict['hits']['hits'][0]['_source']

    numero_processo = processo.get('numeroProcesso', '—')
    classe = processo.get('classe', {}).get('nome', '—')
    data_ajuizamento = processo.get('dataAjuizamento', '—')
    ultima_atualizacao = processo.get('dataHoraUltimaAtualizacao', '—')
    orgao_julgador_data = processo.get('orgaoJulgador', {})
    codigo_orgao_julgador = orgao_julgador_data.get('codigo', '—')
    orgao_julgador = orgao_julgador_data.get('nome', '—')
    Municipio = orgao_julgador_data.get('codigoMunicipioIBGE', '—')

    if processo.get('assuntos') and len(processo['assuntos']) > 0:
        assunto = processo['assuntos'][0].get('nome', '—')
    else:
        assunto = '—'

    print(f'Número do processo: {numero_processo}')
    print(f'Classe: {classe}')
    print(f'Assunto principal: {assunto}')
    print(f'Órgão julgador: {codigo_orgao_julgador} - {orgao_julgador}')
    print(f'Data de ajuizamento: {data_ajuizamento}')
    print(f'Última atualização: {ultima_atualizacao}')
    print(f'Código Município IBGE: {Municipio}')
else:
    print("❌ Nenhum processo encontrado na resposta.")




Número do processo: 00085669120248050256
Classe: Procedimento do Juizado Especial Cível
Assunto principal: Perdas e Danos
Órgão julgador: 17873 - 1ª VARA JUIZADOS ESPECIAIS DE TEIXEIRA DE FREITAS
Data de ajuizamento: 2025-12-16T09:08:36.000Z
Última atualização: 2025-03-20T12:32:07.360Z
Código Município IBGE: 2931350


In [61]:
processos = []

for processo in dados_dict['hits']['hits']:
  numero_processo = processo['_source']['numeroProcesso']
  grau = processo['_source']['grau']
  classe = processo['_source']['classe']['nome']
  assuntos = processo['_source']['assuntos'] # Pode ter mais de um
  data_ajuizamento = processo['_source']['dataAjuizamento']
  ultima_atualizacao = processo['_source']['dataHoraUltimaAtualizacao']
  formato = processo['_source']['formato']['nome']
  codigo = processo['_source']['orgaoJulgador']['codigo']
  orgao_julgador = processo['_source']['orgaoJulgador']['nome']
  municipio = processo['_source']['orgaoJulgador']['codigoMunicipioIBGE']
  sort = processo['sort'][0]
  try:
    movimentos = processo['_source']['movimentos']
  except:
    movimentos = []

  processos.append([numero_processo, classe, data_ajuizamento, ultima_atualizacao, formato, \
                    codigo, orgao_julgador, municipio, grau, assuntos, movimentos, sort])

df = pd.DataFrame(processos, columns=['numero_processo', 'classe', 'data_ajuizamento', 'ultima_atualizacao', \
                      'formato', 'codigo', 'orgao_julgador', 'municipio', 'grau', 'assuntos', 'movimentos', 'sort'])

df.sample(5)

,numero_processo,classe,data_ajuizamento,ultima_atualizacao,formato,codigo,orgao_julgador,municipio,grau,assuntos,movimentos,sort
7422,80090653620248050256,Execução de Título Extrajudicial,2024-09-30T00:00:00.000Z,2025-03-20T13:08:01.401Z,Eletrônico,5274,2ª VARA DE FEITOS DE REL DE CONS. CÍVEL E COME...,2931350,G1,"[{'codigo': 4960, 'nome': 'Cédula de Crédito B...","[{'complementosTabelados': [{'codigo': 3, 'val...",1727654400000
9415,80080260420248050256,Medidas Protetivas de urgência (Lei Maria da P...,2024-08-20T00:00:00.000Z,2025-03-19T21:34:27.525Z,Eletrônico,76526,"2ª VARA CRIME, PRIVATIVA DE VIOLÊNCIA DOMÉSTICA",2931350,G1,"[{'codigo': 10948, 'nome': 'Violência Doméstic...","[{'complementosTabelados': [{'codigo': 4, 'val...",1724112000000
100,80015891020258050256,Auto de Prisão em Flagrante,2025-03-13T00:00:00.000Z,2025-03-20T12:23:24.753Z,Eletrônico,76526,"2ª VARA CRIME, PRIVATIVA DE VIOLÊNCIA DOMÉSTICA",2931350,G1,"[{'codigo': 3608, 'nome': 'Tráfico de Drogas e...","[{'complementosTabelados': [{'codigo': 3, 'val...",1741824000000
2686,00000609220258050256,Procedimento do Juizado Especial Cível,2025-01-10T12:02:32.000Z,2025-03-20T12:26:16.171Z,Eletrônico,18553,2ª VARA JUIZADOS ESPECIAIS DE TEIXEIRA DE FREITAS,2931350,JE,"[{'codigo': 6226, 'nome': 'Inclusão Indevida e...","[{'complementosTabelados': [{'codigo': 2, 'val...",1736510552000
9632,00055710820248050256,Cumprimento de sentença,2024-08-14T09:51:11.000Z,2025-03-20T12:28:34.613Z,Eletrônico,18553,2ª VARA JUIZADOS ESPECIAIS DE TEIXEIRA DE FREITAS,2931350,JE,"[{'codigo': 6226, 'nome': 'Inclusão Indevida e...","[{'complementosTabelados': [{'codigo': 2, 'val...",1723629071000


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


In [62]:
def converte_data(data_str):
    return pd.to_datetime(data_str).tz_convert('America/Sao_Paulo')


def gera_lista_assuntos(assuntos_do_df):
    lst_assuntos=[]
    for assunto in assuntos_do_df:
        try:
            lst_assuntos.append(assunto.get('nome'))
        except:
            lst_assuntos.append('')

    return lst_assuntos


def gera_lista_movimentos(movimentos):
    lst_movimentos_final =[]
    for movimento in movimentos:
        codigo = movimento.get('codigo')
        nome = movimento.get('nome')
        data_hora = movimento.get('dataHora')
        if data_hora:
            data_hora = converte_data(data_hora)
        lst_movimentos_final.append([codigo, nome, data_hora])
    return lst_movimentos_final

In [63]:
def gera_lista_movimentos(movimentos):
    lst_movimentos_final = []
    if isinstance(movimentos, list):
        for movimento in movimentos:
            if isinstance(movimento, dict):  # só processa se for dicionário
                codigo = movimento.get('codigo')
                nome = movimento.get('nome')
                data_hora = movimento.get('dataHora')
                lst_movimentos_final.append((codigo, nome, data_hora))
            else:
                # opcional: registrar ou ignorar listas internas ou itens inválidos
                continue
    return lst_movimentos_final


In [67]:
import pandas as pd

# Exporta a tabela principal para CSV
df.to_csv('anpp_processos.csv', sep=',', index=False, encoding='utf-8')

# Gera o CSV com os movimentos detalhados
movimentos_list = []

for index, row in df.iterrows():
    numero = row.get('numero_processo', '—')
    classe = row.get('classe', '—')
    movimentos = row.get('movimentos', [])

    for mov in movimentos:
        if isinstance(mov, (list, tuple)) and len(mov) >= 3:
            movimentos_list.append({
                "Número do Processo": numero,
                "Classe": classe,
                "Data do Movimento": mov[2],
                "Código": mov[0],
                "Descrição": mov[1]
            })
        elif isinstance(mov, dict):
            movimentos_list.append({
                "Número do Processo": numero,
                "Classe": classe,
                "Data do Movimento": mov.get('dataHora', '—'),
                "Código": mov.get('codigo', '—'),
                "Descrição": mov.get('nome', '—')
            })

# Cria o DataFrame de movimentos
df_mov = pd.DataFrame(movimentos_list)

# Exporta para CSV
df_mov.to_csv('anpp_movimentos.csv', sep=',', index=False, encoding='utf-8')

